In [1]:
import cv2
import numpy as np
import pandas as pd

## Image Domain의 점을 Hough Domain(rho, theta)에서 선으로 만들고 가장 많이 교차하는 점을 구한다.
def make_Hough_Domain(img):                  
    global img_edge                                           ## 영상 출력하기 위해 전역변수 선언
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)           
    img_edge = cv2.GaussianBlur(img_gray, (5, 5), 50, 50)      ## noise를 줄이기 위해서 가우시안으로 Blur를 취하고
    img_edge = cv2.Canny(img_edge, 5, 8, 3)                    ## Edge를 검출한다.
    

    ## Hough Domain의 경우 0~180까지의 theta값이 있기 때문에 Rho가 음수 값이 나올 수 있다. 
    domain = np.zeros((height*3, 180), np.uint8)               
    
    for y in range(0, height):
        for x in range(0, width):
            ## Edge만 남은 binary 이미지에서 양수 값을 가지는 pixel만 Hough Domain의 선으로 Transform한다.
            if img_edge.item(y, x) > 0:
                for theta in range(0,180):
                    ## 모든 Rho를 Domain에 찍기 위해서 Rho에 Rho의 최저값을 양수로 더해준다. ( Rho = x*cos()+y*sin() )
                    ## 찍히는 pixel마다 +1하여 Hough_Transform 함수에서 최대값을 가지는 pixel 선별한다.
                    rho = int(x * np.cos(np.radians(theta)) + y * np.sin(np.radians(theta)))
                    rho = rho + max(height, width)
                    domain[rho, theta] +=1
    
    return domain

## Hough Domain(선별된 점) -> Image Domain(선) 으로 Transform 시킨다.
def Hough_Transform(img, hough_domain , hough_elect):
    max_value=[]
    index=[]
    
    for i in range(0, len(hough_domain)) :             ## 각 행에서 최대값과 pixel의 좌표를 추출한다.
        max_value.append(np.max(hough_domain[i]))
        temp = np.argmax(hough_domain[i])
        index.append([i,temp])
    
    ## 추출된  최대값은 제일 많이 찍힌 pixel로서, 이는 Hough Domain에서 교차한 횟수가 가장 많은 pixel을 뜻한다.
    ## 최대값으로 rank를 세우고, 상위 5개의 점을 선별한다. (img에 따라 다르지만 보통 5-10개 사이)
    df = pd.DataFrame( {'max_value': max_value, 'index': index})
    df['rank_max'] = df['max_value'].rank(method='max', ascending=False)
    df = df[df['rank_max'] <= 5]
    
    ## 최대값을 가지는 pixel의 좌표를 가지고 선별된 pixel을 파란색으로 출력한다.
    ## Hough Domain에서 선을 나타내기 위해서 아래와 같이 조건문을 설정했다.
    ## x1,x2를 각각 pixel의 수평 방향 끝(0, width)로 놓고, y 값이 변하면 theta값 45에서 135 사이의 선을 표현할 수 있다.
    ## y1,y2를 각각 pixel의 수직 방향 끝(0, height)로 놓고, x 값이 변하면 theta값 0~45, 135~180 사이의 선을 표현할 수 있다.
    ## 각 좌표 식은 Rho = x*cos()+y*sin()에 기초하나, Rho에 더해준 값이 있으므로 
    ## Rho = x*cos() + y*sin() + Alpha 식으로 x,y에 대해 정리하여 계산한다.
    for r, th in df['index'].values :
        hough_elect.itemset(r, th, 0, 0)
        hough_elect.itemset(r, th, 1, 0)
        hough_elect.itemset(r, th, 2, 255)
        
        if th >= 45 and th <= 135:
            x1 = 0
            x2 = width
            y1 = int(((r - max(height,width)) - x1 * np.cos(np.radians(th))) / np.sin(np.radians(th)))
            y2 = int(((r - max(height,width)) - x2 * np.cos(np.radians(th))) / np.sin(np.radians(th)))

        else:
            y1 = 0
            y2 = height
            x1 = int(((r - max(height,width)) - y1 * np.sin(np.radians(th))) / np.cos(np.radians(th)))
            x2 = int(((r - max(height,width)) - y2 * np.sin(np.radians(th))) / np.cos(np.radians(th)))
        
        cv2.line(img, (x1,y1), (x2,y2), (0,0,255), 1)
    
    return img, hough_elect


if __name__ == "__main__":
    img = cv2.imread('opencv/picture/1.jpg', cv2.IMREAD_COLOR)
    img = cv2.resize(img, (300,300))
    
    global height, width
    height, width = img.shape[0:2]
    
    hough_domain = make_Hough_Domain(img)
    hough_elect = np.zeros((hough_domain.shape[0], hough_domain.shape[1], 3), np.uint8)

    img_result, img_hough_elect = Hough_Transform(img, hough_domain, hough_elect)

    cv2.imshow("Edge Display", img_edge)
    cv2.imshow("Hough D", hough_domain)
    cv2.imshow("Hough elect", img_hough_elect)
    cv2.imshow("result", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [19]:
import cv2 
import numpy as np

img = cv2.imread('opencv/picture/1.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

## cv2.Canny(이미지, threshold1, threshold2)
edges = cv2.Canny(gray,0,250)

##cv2.HoughLinesP(이미지, rho값의 범위, theta값의 범위, threshold값, 선의 최소길이, 선과 선사이 허용간격)
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 200, minLineLength=100, maxLineGap=10)

for line in lines:
    x1,y1,x2,y2 = line[0]
    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)

cv2.imshow("result", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
import cv2
import numpy as np

img = cv2.imread('opencv/picture/1.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

## cv2.Canny(이미지, threshold1, threshold2)
edges = cv2.Canny(gray,500,1000)

##cv2.HoughLines(이미지, rho값의 범위, theta값의 범위, threshold값)
lines = cv2.HoughLines(edges,1,np.pi/180,200)

for line in lines:
    rho,theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

cv2.imshow("result", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [29]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
